In [2]:
import os
import glob
import math
import pickle

import numpy as np
import pandas as pd
import geopandas as gpd
import torch

import processing_io as pio
from torch_geometric.transforms import LineGraph

from torch_geometric.data import Data, Batch
import shapely.wkt as wkt

base_dir = '../../../../data/pop_1pm_simulations/sample_sim_output/'

In [3]:
eqasim_pt = pd.read_csv(base_dir + 'eqasim_pt.csv', sep = ';')
eqasim_trips = pd.read_csv(base_dir + 'eqasim_trips.csv', sep = ';')
output_links = pd.read_csv(base_dir + "output_links.csv", sep = ';')

In [4]:
# compute, for eqasim_trips, the travel_time and the routed_distance per mode.

# Group by mode and calculate total travel time and routed distance
mode_stats = eqasim_trips.groupby('mode').agg({
    'travel_time': 'sum',
    'routed_distance': 'sum'
}).reset_index()

# Rename columns for clarity
mode_stats.columns = ['mode', 'total_travel_time', 'total_routed_distance']

# Display the results
print(mode_stats)

            mode  total_travel_time  total_routed_distance
0           bike           448127.0           1.389817e+06
1            car          1598811.0           1.799725e+07
2  car_passenger           360236.0           4.076165e+06
3        outside             2175.0           2.916280e+06
4             pt          6458371.0           2.199990e+07
5           walk          3247065.0           3.898260e+06


In [5]:
eqasim_trips.head()

,person_id,person_trip_id,origin_x,origin_y,destination_x,destination_y,departure_time,travel_time,vehicle_distance,routed_distance,mode,preceding_purpose,following_purpose,returning,euclidean_distance
0,7826870,0,650920.884420,6.863808e+06,655238.176446,6.866969e+06,0.0,815.0,10480.891994,10549.136691,car,work,outside,False,5350.733594
1,5800773,0,655523.682149,6.860831e+06,652740.495115,6.857491e+06,14389.0,560.0,5950.920874,5958.305713,car,work,outside,False,4347.092640
2,10583393,0,643494.978291,6.863251e+06,653170.349957,6.857549e+06,14717.0,861.0,14246.678480,14246.678480,car,outside,outside,False,11230.555749
3,10583393,1,653170.349957,6.857549e+06,653412.473013,6.857613e+06,15578.0,0.0,0.000000,250.437965,outside,outside,outside,False,250.437965
4,198751,0,649511.669695,6.859818e+06,647125.109579,6.861556e+06,13743.0,1843.0,3393.799120,4209.407269,pt,home,work,False,2952.550515


In [6]:
output_links.head()

,link,from_node,to_node,length,freespeed,capacity,lanes,modes,vol_car,osm:relation:route_master,...,osm:way:id,osm:way:access,osm:way:oneway,osm:way:highway,osm:relation:route,osm:way:railway,osm:way:name,storageCapacityUsedInQsim,osm:way:tunnel,geometry
0,100315,24972409,24972408,16.181257,8.333333,480.0,1.0,"bus,car,car_passenger",3,NaN,...,4216830.0,NaN,yes,residential,bicycle,NaN,Carrefour de l'Odéon,NaN,NaN,LINESTRING( 651473.3041634131 6861511.11045421...
1,100316,5904976363,24983651,14.860209,8.333333,480.0,1.0,"bus,car,car_passenger,pt",11,NaN,...,4216831.0,NaN,NaN,tertiary,bus,NaN,Carrefour de l'Odéon,NaN,NaN,LINESTRING( 651477.8178584711 6861578.65907928...
2,100317,24983651,5904976363,14.860209,8.333333,960.0,2.0,"bus,car,car_passenger,pt",1,NaN,...,4216831.0,NaN,NaN,tertiary,bus,NaN,Carrefour de l'Odéon,NaN,NaN,LINESTRING( 651475.6766977471 6861563.95433103...
3,100321,664205947,24972376,22.264540,8.333333,960.0,2.0,"car,car_passenger",5,NaN,...,4216834.0,NaN,yes,residential,NaN,NaN,Boulevard Saint-Germain,NaN,NaN,LINESTRING( 651565.0784184289 6861531.47025577...
4,100324,24972376,24972375,64.853276,8.333333,480.0,1.0,"bus,car,car_passenger",5,NaN,...,4216833.0,NaN,yes,residential,bicycle,NaN,Rue Dupuytren,NaN,NaN,"LINESTRING( 651559.387416067 6861509.94533646,..."
